# Classify Voice Clips on Features
### In this notebook, we will build classifiers to classify voice clips on the features that have been extracted. The following features are extracted:
1. Zero Crossing Rate : The rate of sign-changes of the signal during the duration of a particular frame.
2. Chroma STFT (Short-Time Fourier Transform): Refers to the chroma feature representation derived from the short-time Fourier transform of an audio signal. Chroma features, or chromagrams, represent the energy distribution among the twelve different pitch classes (C, C#, D, ..., B) of the musical octave. .
3. Mel Spectrogram: A Mel spectrogram is a representation of the power spectrum of a sound signal, where the frequencies are converted to the Mel scale. The Mel scale is designed to mimic the human ear's perception of sound, where each Mel unit corresponds to a perceived equal step in pitch.
4.  MFCC: Mel Frequency Cepstral Coefficients form a cepstral representation where the frequency bands are not linear but distributed according to the mel-scale.
5. RMS: root-mean-square (RMS) value for each frame, either from the audio samples or from a spectrogram.
6. Chroma CQT: Constant-Q chromagram
7. Chroma CENS: Chroma variant “Chroma Energy Normalized” (CENS)
8. Chroma VQT: Variable-Q chromagram
9. Spectral Centroid : The center of gravity of the spectrum.
10. Spectral Bandwidth: Compute pth-order spectral bandwidth..
11. Spectral Contrast :  Compute spectral contrast.
12. Spectral Flatness: Compute spectral flatness
13. Spectral Rolloff : The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and the bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent to a value close to 1 (or 0).

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load the Data and Features

In [ ]:
data_features_path = A_PATH

test_data_features_path = A_PATH


In [ ]:
all_data_features = pd.read_csv(data_features_path)

test_data_features = pd.read_csv(test_data_features_path)

In [ ]:
all_data_features.shape, test_data_features.shape

In [ ]:
all_data_features.columns

In [ ]:
from itertools import combinations

def generate_subsets_min_length(lst, min_length):
    subsets = []
    for i in range(min_length, len(lst) + 1):
        subsets.extend(combinations(lst, i))
    return [list(subset) for subset in subsets]

In [ ]:
features_list = ['zcrate_mean',
       'chroma_stft_mean', 'melspectrogram_mean', 'mfcc_feature', 'rms_mean',
       'chroma_cqt_mean', 'chroma_cens_mean', 'chroma_vqt_mean', 'spcent_mean',
       'spband_mean', 'spcontrast_mean', 'spflat_mean', 'sprolloff_mean']

In [ ]:
len(features_list)

In [ ]:
all_features_combs = generate_subsets_min_length(features_list, 2)

In [ ]:
len(all_features_combs)

In [ ]:
all_features_combs = generate_subsets_min_length(features_list, 1)
len(all_features_combs)

## Test on Different Sets of Features

In [ ]:
all_data_features.columns

In [ ]:
from xgboost import XGBClassifier

## With 1 Feature and Hyperparameter Tuning

In [ ]:
features_list_list = [[feature] for feature in features_list]
features_list_list

In [ ]:
## on various feature combinations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import ast

records = []

features_combs = features_list_list

for features_comb in tqdm(features_combs):

    columns_to_use = ['label']
    columns_to_use.extend(features_comb)

    data = all_data_features[columns_to_use]

    test_data = test_data_features[columns_to_use]


    data = data.copy()

    col_to_convert = []
    for acol in features_comb:
        if data[acol].dtype == 'object':
            col_to_convert.append(acol)

    # Create a list to hold the new DataFrames
    new_cols_df = []

    for col in col_to_convert:

        data[col] = data[col].apply(ast.literal_eval)

        ## Flatten the list to make individual columns for each individual elements

        # Create a DataFrame with the new columns
        new_col_df = pd.DataFrame(data[col].tolist(), index=data.index)
        new_col_df.columns = [f"{col}_{idx}" for idx in new_col_df.columns]

        # Append the new DataFrame to the list
        new_cols_df.append(new_col_df)

    # Concatenate the original DataFrame with the new columns
    new_data = pd.concat([data] + new_cols_df, axis=1)

    # Drop the original string columns
    new_data = new_data.drop(columns=col_to_convert)

    ### Do the same for test data

    test_data = test_data.copy()

    # Create a list to hold the new DataFrames
    new_test_cols_df = []

    for col in col_to_convert:

        test_data[col] = test_data[col].apply(ast.literal_eval)

        ## Flatten the list to make individual columns for each individual elements

        # Create a DataFrame with the new columns
        new_test_col_df = pd.DataFrame(test_data[col].tolist(), index=test_data.index)
        new_test_col_df.columns = [f"{col}_{idx}" for idx in new_test_col_df.columns]

        # Append the new DataFrame to the list
        new_test_cols_df.append(new_test_col_df)

    # Concatenate the original DataFrame with the new columns
    new_test_data = pd.concat([test_data] + new_test_cols_df, axis=1)

    # Drop the original string columns
    new_test_data = new_test_data.drop(columns=col_to_convert)


    ## Classify by Logistic Regression

    X = new_data.drop(columns=['label']).values
    y = np.array(new_data['label'].map({'engaging':0, 'boring':1}))

    X_val = new_test_data.drop(columns=['label']).values
    y_val = np.array(new_test_data['label'].map({'engaging':0, 'boring':1}))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # make train as the whole set
    X_train = X
    y_train = y

    sc = StandardScaler()
    sc.fit(X_train)

    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    X_val_std = sc.transform(X_val)

    ## hyperparameter tuning on logistic regression
    C_list = [0.01, 0.1, 1, 10, 100]

    accuracy_max = 0
    C_max = 0
    lr_max = None
    for C in C_list:
        lr = LogisticRegression(C=C, random_state=42, solver='lbfgs', max_iter=500)
        lr.fit(X_train_std, y_train)
        # predict the test data
        y_val_pred = lr.predict(X_val_std)
        accuracy_C = accuracy_score(y_val, y_val_pred)
        if accuracy_C > accuracy_max:
            accuracy_max = accuracy_C
            C_max = C
            lr_max = lr

    ## Use the best hyperparameter C
    #lr = LogisticRegression(C=C_max, random_state=42, solver='lbfgs', max_iter=500)

    #lr.fit(X_train_std, y_train)

    # predict the data
    y_pred = lr_max.predict(X_test_std)


    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'logistic regression with C_max={}'.format(C_max)
    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # predict the test data
    y_val_pred = lr_max.predict(X_val_std)

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'logistic regression with C_max={}'.format(C_max)
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    ## Classify by Random Forest"""

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

    n_estimators_list = [200, 500]
    min_samples_split_list = [2, 10]

    accuracy_max = 0
    n_estimators_max = 0
    min_samples_split_max = 0
    rf_max = None
    for n_estimators in n_estimators_list:
        for min_samples_split in min_samples_split_list:
            # Initialize the classifier
            rf_classifier = RandomForestClassifier(n_estimators=n_estimators,
                                                   min_samples_split=min_samples_split,
                                                   random_state=42)

            # Train the classifier
            rf_classifier.fit(X_train, y_train)
            # predict the test data
            y_val_pred = rf_classifier.predict(X_val_std)
            accuracy_tuning = accuracy_score(y_val, y_val_pred)
            if accuracy_tuning > accuracy_max:
                accuracy_max = accuracy_tuning
                n_estimators_max = n_estimators
                min_samples_split_max = min_samples_split
                rf_max = rf_classifier


    #rf_classifier = RandomForestClassifier(n_estimators=n_estimators_max, min_samples_split =
                                           #min_samples_split_max, random_state=42)

    # Train the classifier
    #rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_max.predict(X_test)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_test, y_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'random forest with n_estimators={} and min_samples_split={}'.format(
        n_estimators_max, min_samples_split_max
    )

    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # Make predictions on individual set
    y_val_pred = rf_max.predict(X_val)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_val, y_val_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'random forest with n_estimators={} and min_samples_split={}'.format(
        n_estimators_max, min_samples_split_max
    )
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    ## Classify by XGBoost"""

    from xgboost import XGBClassifier

    n_estimators_list = [200, 500]
    learning_rate_list = [0.01, 0.2]

    accuracy_max = 0
    n_estimators_max = 0
    learning_rate_max = 0
    xgb_max = None
    for n_estimators in n_estimators_list:
        for learning_rate in learning_rate_list:
            # Initialize the classifier
            xgb_classifier = XGBClassifier(n_estimators=n_estimators,
                                           learning_rate=learning_rate,
                                           random_state=42)

            # Train the classifier
            xgb_classifier.fit(X_train, y_train)
            # predict the test data
            y_val_pred = xgb_classifier.predict(X_val_std)
            accuracy_tuning = accuracy_score(y_val, y_val_pred)
            if accuracy_tuning > accuracy_max:
                accuracy_max = accuracy_tuning
                n_estimators_max = n_estimators
                learning_rate_max = learning_rate
                xgb_max = xgb_classifier

    # Initialize the classifier
    #xgb_classifier = XGBClassifier(n_estimators=n_estimators_max,
                                   #learning_rate=learning_rate_max,
                                   #random_state=42)

    # Train the classifier
    #xgb_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = xgb_max.predict(X_test)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_test, y_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'XGBoost with n_estimators={} and learning_rate={}'.format(
        n_estimators_max, learning_rate_max
    )
    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # Make predictions on individual set
    y_val_pred = xgb_max.predict(X_val)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_val, y_val_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'XGBoost with n_estimators={} and learning_rate={}'.format(
        n_estimators_max, learning_rate_max
    )
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    if len(records) % 40 == 0:
        evals = pd.DataFrame(records)
        evals.to_csv("test_evaluation_1_feature_results.csv", index=None)

evals = pd.DataFrame(records)
evals.to_csv("test_evaluation_1_feature_results.csv", index=None)

In [ ]:
evals

In [ ]:
f1_boring_max = evals[evals.evaluation_data == 'individual set'].f1_boring.max()

In [ ]:
evals[evals.f1_boring == f1_boring_max]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
evals[evals.evaluation_data == 'individual set'].sort_values(by='f1_boring', ascending=False)

## On At Least 2 Feature Combinations

In [ ]:
## on various feature combinations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import ast

records = []

features_combs = generate_subsets_min_length(features_list, 2)

for features_comb in tqdm(features_combs):

    columns_to_use = ['label']
    columns_to_use.extend(features_comb)

    data = all_data_features[columns_to_use]

    test_data = test_data_features[columns_to_use]


    data = data.copy()

    col_to_convert = []
    for acol in features_comb:
        if data[acol].dtype == 'object':
            col_to_convert.append(acol)

    # Create a list to hold the new DataFrames
    new_cols_df = []

    for col in col_to_convert:

        data[col] = data[col].apply(ast.literal_eval)

        ## Flatten the list to make individual columns for each individual elements

        # Create a DataFrame with the new columns
        new_col_df = pd.DataFrame(data[col].tolist(), index=data.index)
        new_col_df.columns = [f"{col}_{idx}" for idx in new_col_df.columns]

        # Append the new DataFrame to the list
        new_cols_df.append(new_col_df)

    # Concatenate the original DataFrame with the new columns
    new_data = pd.concat([data] + new_cols_df, axis=1)

    # Drop the original string columns
    new_data = new_data.drop(columns=col_to_convert)

    ### Do the same for test data

    test_data = test_data.copy()

    # Create a list to hold the new DataFrames
    new_test_cols_df = []

    for col in col_to_convert:

        test_data[col] = test_data[col].apply(ast.literal_eval)

        ## Flatten the list to make individual columns for each individual elements

        # Create a DataFrame with the new columns
        new_test_col_df = pd.DataFrame(test_data[col].tolist(), index=test_data.index)
        new_test_col_df.columns = [f"{col}_{idx}" for idx in new_test_col_df.columns]

        # Append the new DataFrame to the list
        new_test_cols_df.append(new_test_col_df)

    # Concatenate the original DataFrame with the new columns
    new_test_data = pd.concat([test_data] + new_test_cols_df, axis=1)

    # Drop the original string columns
    new_test_data = new_test_data.drop(columns=col_to_convert)


    ## Classify by Logistic Regression

    X = new_data.drop(columns=['label']).values
    y = np.array(new_data['label'].map({'engaging':0, 'boring':1}))

    X_val = new_test_data.drop(columns=['label']).values
    y_val = np.array(new_test_data['label'].map({'engaging':0, 'boring':1}))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # make train as the whole set
    X_train = X
    y_train = y

    sc = StandardScaler()
    sc.fit(X_train)

    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    X_val_std = sc.transform(X_val)

    ## hyperparameter tuning on logistic regression
    C_list = [0.01, 0.1, 1, 10, 100]

    accuracy_max = 0
    C_max = 0
    lr_max = None
    for C in C_list:
        lr = LogisticRegression(C=C, random_state=42, solver='lbfgs', max_iter=500)
        lr.fit(X_train_std, y_train)
        # predict the test data
        y_val_pred = lr.predict(X_val_std)
        accuracy_C = accuracy_score(y_val, y_val_pred)
        if accuracy_C > accuracy_max:
            accuracy_max = accuracy_C
            C_max = C
            lr_max = lr

    ## Use the best hyperparameter C
    #lr = LogisticRegression(C=C_max, random_state=42, solver='lbfgs', max_iter=500)

    #lr.fit(X_train_std, y_train)

    # predict the data
    y_pred = lr_max.predict(X_test_std)


    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'logistic regression with C_max={}'.format(C_max)
    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # predict the test data
    y_val_pred = lr_max.predict(X_val_std)

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'logistic regression with C_max={}'.format(C_max)
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    ## Classify by Random Forest"""

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

    n_estimators_list = [200, 500]
    min_samples_split_list = [2, 10]

    accuracy_max = 0
    n_estimators_max = 0
    min_samples_split_max = 0
    rf_max = None
    for n_estimators in n_estimators_list:
        for min_samples_split in min_samples_split_list:
            # Initialize the classifier
            rf_classifier = RandomForestClassifier(n_estimators=n_estimators,
                                                   min_samples_split=min_samples_split,
                                                   random_state=42)

            # Train the classifier
            rf_classifier.fit(X_train, y_train)
            # predict the test data
            y_val_pred = rf_classifier.predict(X_val_std)
            accuracy_tuning = accuracy_score(y_val, y_val_pred)
            if accuracy_tuning > accuracy_max:
                accuracy_max = accuracy_tuning
                n_estimators_max = n_estimators
                min_samples_split_max = min_samples_split
                rf_max = rf_classifier


    #rf_classifier = RandomForestClassifier(n_estimators=n_estimators_max, min_samples_split =
    #                                      min_samples_split_max, random_state=42)

    # Train the classifier
    #rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_max.predict(X_test)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_test, y_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'random forest with n_estimators={} and min_samples_split={}'.format(
        n_estimators_max, min_samples_split_max
    )

    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # Make predictions on individual set
    y_val_pred = rf_max.predict(X_val)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_val, y_val_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'random forest with n_estimators={} and min_samples_split={}'.format(
        n_estimators_max, min_samples_split_max
    )
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    ## Classify by XGBoost"""

    from xgboost import XGBClassifier

    n_estimators_list = [200, 500]
    learning_rate_list = [0.01, 0.2]

    accuracy_max = 0
    n_estimators_max = 0
    learning_rate_max = 0
    xgb_max = None
    for n_estimators in n_estimators_list:
        for learning_rate in learning_rate_list:
            # Initialize the classifier
            xgb_classifier = XGBClassifier(n_estimators=n_estimators,
                                           learning_rate=learning_rate,
                                           random_state=42)

            # Train the classifier
            xgb_classifier.fit(X_train, y_train)
            # predict the test data
            y_val_pred = xgb_classifier.predict(X_val_std)
            accuracy_tuning = accuracy_score(y_val, y_val_pred)
            if accuracy_tuning > accuracy_max:
                accuracy_max = accuracy_tuning
                n_estimators_max = n_estimators
                learning_rate_max = learning_rate
                xgb_max = xgb_classifier

    # Initialize the classifier
    #xgb_classifier = XGBClassifier(n_estimators=n_estimators_max,
    #                               learning_rate=learning_rate_max,
    #                               random_state=42)

    # Train the classifier
    #xgb_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = xgb_max.predict(X_test)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_test, y_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_test, y_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_test, y_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'XGBoost with n_estimators={} and learning_rate={}'.format(
        n_estimators_max, learning_rate_max
    )
    rec['evaluation_data'] = 'split from training'
    rec['accuracy'] = accuracy_score(y_test, y_pred)
    rec['precision_boring'] = precision_score(y_test, y_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_test, y_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_test, y_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_test, y_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_test, y_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_test, y_pred, pos_label=0)
    records.append(rec)

    # Make predictions on individual set
    y_val_pred = xgb_max.predict(X_val)

    # Evaluate the predictions
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_val, y_val_pred))

    #print("\nClassification Report:")
    #print(classification_report(y_val, y_val_pred))

    #print("\nAccuracy Score:")
    #print(accuracy_score(y_val, y_val_pred))

    rec = {}
    rec['features'] = features_comb
    rec['model'] = 'XGBoost with n_estimators={} and learning_rate={}'.format(
        n_estimators_max, learning_rate_max
    )
    rec['evaluation_data'] = 'individual set'
    rec['accuracy'] = accuracy_score(y_val, y_val_pred)
    rec['precision_boring'] = precision_score(y_val, y_val_pred, pos_label=1)
    rec['recall_boring'] = recall_score(y_val, y_val_pred, pos_label=1)
    rec['f1_boring'] = f1_score(y_val, y_val_pred, pos_label=1)
    rec['precision_engaging'] = precision_score(y_val, y_val_pred, pos_label=0)
    rec['recall_engaging'] = recall_score(y_val, y_val_pred, pos_label=0)
    rec['f1_engaging'] = f1_score(y_val, y_val_pred, pos_label=0)
    records.append(rec)

    if len(records) % 40 == 0:
        evals = pd.DataFrame(records)
        evals.to_csv("test_evaluation_2_features_results.csv", index=None)

evals = pd.DataFrame(records)
evals.to_csv("test_evaluation_2_features_results.csv", index=None)

In [ ]:
evals

In [ ]:
f1_boring_max = evals[evals.evaluation_data == 'individual set'].f1_boring.max()

In [ ]:
evals[evals.f1_boring == f1_boring_max]

In [ ]:
evals[evals.evaluation_data == 'individual set'].sort_values(by='f1_boring', ascending=False)

In [ ]:
evals[(evals.evaluation_data == 'individual set') & (evals.model.str.contains("random forest"))
                                                       ].sort_values(by='f1_boring', ascending=False)

In [ ]:
data.columns

In [ ]:
data[['zcrate_mean', 'chroma_cqt_mean', 'spcent_mean', 'spband_mean']]

## Combine 1-feature and 2-features Results

In [ ]:
test_1_feature_results = pd.read_csv("test_evaluation_1_feature_results.csv")
test_1_feature_results.shape

In [ ]:
test_2_features_results = pd.read_csv("test_evaluation_2_features_results.csv")
test_2_features_results.shape

In [ ]:
test_all_results = pd.concat([test_1_feature_results, test_2_features_results], ignore_index=True)
test_all_results.shape

In [ ]:
test_all_results.to_csv("test_evaluation_all_results.csv", index=None)

In [ ]:
evals[(evals.evaluation_data == 'individual set')].sort_values(by='f1_boring', ascending=False)

In [ ]:
evals[(evals.evaluation_data == 'individual set') & (evals.model.str.contains("random forest"))
                                                       ].sort_values(by='f1_boring', ascending=False)

In [ ]:
evals[(evals.evaluation_data == 'individual set') & (evals.model.str.contains("XGBoost"))
                                                       ].sort_values(by='f1_boring', ascending=False)